<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/XGBoost_for_Revenue_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Necessary Libraries

In [1]:
!pip install xgboost

import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#3. Load the Data

In [ ]:
train_path='/content/drive/MyDrive/VERGER/Revenue_Forecasting/train_forecast_data.csv'
test_path='/content/drive/MyDrive/VERGER/Revenue_Forecasting/test_forecast_data.csv'

traib_df=pd.read_csv(train_path)
test_df=pd.read_csv(test_path)

#4. Select Features and Target

In [ ]:
features = ['Account', 'Product', 'Type',
            'Lag_Qty_1', 'Lag_Qty_2', 'Lag_Rev_1', 'Lag_Rev_2',
            'Rolling_Qty_3', 'Rolling_Rev_3']

target = 'Forecast_Revenue'

# Drop rows with missing values
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]


#5. Setup Preprocessing Pipeline

In [ ]:
categorical_features = ['Account', 'Product', 'Type']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, categorical_features)],
    remainder='passthrough'
)

#6. Create XGBoost Pipeline

In [ ]:
from xgboost import XGBRegressor

xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    ))
])

#7. train the Model

In [ ]:
xgb_model.fit(X_train, y_train)

#8. Evaluate the Model

In [ ]:
y_pred = xgb_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n📊 XGBoost Revenue Forecasting Results:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE : {mae:.2f}")
print(f"  R²  : {r2:.4f}")